In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from collections import defaultdict
from scipy.stats.stats import pearsonr
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram

In [2]:
df = pd.read_csv('training.csv')

In [3]:
df['MMRAcquisitionAuctionAveragePrice'].fillna(df['MMRAcquisitionAuctionAveragePrice'].median(), inplace = True)
df['MMRAcquisitionAuctionCleanPrice'].fillna(df['MMRAcquisitionAuctionCleanPrice'].median(), inplace = True)
df['MMRAcquisitionRetailAveragePrice'].fillna(df['MMRAcquisitionRetailAveragePrice'].median(), inplace = True)
df['MMRAcquisitonRetailCleanPrice'].fillna(df['MMRAcquisitonRetailCleanPrice'].median(), inplace = True)
df['MMRCurrentAuctionAveragePrice'].fillna(df['MMRCurrentAuctionAveragePrice'].median(), inplace = True)
df['MMRCurrentAuctionCleanPrice'].fillna(df['MMRCurrentAuctionCleanPrice'].median(), inplace = True)
df['MMRCurrentRetailAveragePrice'].fillna(df['MMRCurrentRetailAveragePrice'].median(), inplace = True)
df['MMRCurrentRetailCleanPrice'].fillna(df['MMRCurrentRetailCleanPrice'].median(), inplace = True)

In [4]:
df['MMR_factor'] = ( df['MMRAcquisitionAuctionAveragePrice'] + df['MMRAcquisitonRetailCleanPrice'] + df['MMRCurrentAuctionCleanPrice'] + df['MMRCurrentRetailAveragePrice']) / 4

In [5]:
#BINNING NUMERICAL VALUES
#We have to use the sturges' rule but that gives us 17 bin but i think 10 is better
df['VehBCost_bin'] = pd.cut(df.VehBCost.astype(int), 10, right = False) 
df['VehOdo_bin'] = pd.cut(df.VehOdo.astype(int), 10, right = False)
df['MMR_factor_bin'] = pd.cut(df.MMR_factor.astype(int), 10, right = False)
df['WarrantyCost_bin'] = pd.cut(df.WarrantyCost.astype(int), 10, right = False)

In [6]:
#SELECT THE COLUMNS I AM INTERESTED IN
df_new = df[['Auction','VehicleAge','Make','Trim','Color','Transmission','WheelType','Nationality','Size',
            'TopThreeAmericanName','VNZIP1','VNST','VehBCost_bin', 'VehOdo_bin', 'MMR_factor_bin', 'WarrantyCost_bin']]

In [7]:
#FIX MISSING VALUES
df_new.isnull().sum()

Auction                    0
VehicleAge                 0
Make                       0
Trim                    1911
Color                      7
Transmission               8
WheelType               2577
Nationality                4
Size                       4
TopThreeAmericanName       4
VNZIP1                     0
VNST                       0
VehBCost_bin               0
VehOdo_bin                 0
MMR_factor_bin             0
WarrantyCost_bin           0
dtype: int64

In [8]:
df_new.dropna(subset = ['Transmission','Color','Nationality','Size','TopThreeAmericanName'], inplace=True)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_new.Trim.fillna('Bas', inplace = True, limit=801)

C:\Users\bonin\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
df_new.Trim.fillna('LS', inplace = True, limit=580)

In [11]:
df_new.Trim.fillna('SE', inplace = True, limit=530)

In [12]:
df_new.WheelType.fillna('Alloy', inplace = True, limit=1377)

In [13]:
df_new.WheelType.fillna('Covers', inplace = True, limit=1200)

In [14]:
df_new.Color.isnull().sum()
df_new.Transmission.isnull().sum()
df_new.Nationality.isnull().sum()
df_new.Size.isnull().sum()
df_new.TopThreeAmericanName.isnull().sum()
df_new.Trim.isnull().sum()
df_new.WheelType.isnull().sum()

0

In [15]:
df_new.isnull().sum()

Auction                 0
VehicleAge              0
Make                    0
Trim                    0
Color                   0
Transmission            0
WheelType               0
Nationality             0
Size                    0
TopThreeAmericanName    0
VNZIP1                  0
VNST                    0
VehBCost_bin            0
VehOdo_bin              0
MMR_factor_bin          0
WarrantyCost_bin        0
dtype: int64

In [16]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,Transmission,WheelType,Nationality,Size,TopThreeAmericanName,VNZIP1,VNST,VehBCost_bin,VehOdo_bin,MMR_factor_bin,WarrantyCost_bin
0,ADESA,2,KIA,EX,BLUE,AUTO,Covers,OTHER ASIAN,MEDIUM,OTHER,33619,FL,"[3649.4, 7297.8)","[60271.0, 71360.2)","[7598.4, 11397.6)","[462.0, 1165.6)"
1,MANHEIM,3,DODGE,SXT,SILVER,AUTO,Covers,AMERICAN,MEDIUM,CHRYSLER,32124,FL,"[3649.4, 7297.8)","[82449.4, 93538.6)","[3799.2, 7598.4)","[1165.6, 1869.2)"
2,OTHER,2,DODGE,SE,BLACK,AUTO,Covers,AMERICAN,MEDIUM,CHRYSLER,74135,OK,"[7297.8, 10946.2)","[38092.6, 49181.8)","[7598.4, 11397.6)","[462.0, 1165.6)"
3,OTHER,4,FORD,S,BROWN,AUTO,Covers,AMERICAN,VAN,FORD,85018,AZ,"[3649.4, 7297.8)","[71360.2, 82449.4)","[3799.2, 7598.4)","[1165.6, 1869.2)"
4,MANHEIM,4,CHRYSLER,Bas,BLUE,AUTO,Alloy,AMERICAN,VAN,CHRYSLER,75236,TX,"[3649.4, 7297.8)","[71360.2, 82449.4)","[3799.2, 7598.4)","[1165.6, 1869.2)"


In [17]:
#this code is necessary because if we don't add a string to the algorithm value..
#..when we run the algorithm it doesn't find the attribute values
for i in df_new:
        df_new[i] = df_new[i].astype(str) + "_{}".format(i)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [18]:
df_new.head()

,Auction,VehicleAge,Make,Trim,Color,Transmission,WheelType,Nationality,Size,TopThreeAmericanName,VNZIP1,VNST,VehBCost_bin,VehOdo_bin,MMR_factor_bin,WarrantyCost_bin
0,ADESA_Auction,2_VehicleAge,KIA_Make,EX_Trim,BLUE_Color,AUTO_Transmission,Covers_WheelType,OTHER ASIAN_Nationality,MEDIUM_Size,OTHER_TopThreeAmericanName,33619_VNZIP1,FL_VNST,"[3649.4, 7297.8)_VehBCost_bin","[60271.0, 71360.2)_VehOdo_bin","[7598.4, 11397.6)_MMR_factor_bin","[462.0, 1165.6)_WarrantyCost_bin"
1,MANHEIM_Auction,3_VehicleAge,DODGE_Make,SXT_Trim,SILVER_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,CHRYSLER_TopThreeAmericanName,32124_VNZIP1,FL_VNST,"[3649.4, 7297.8)_VehBCost_bin","[82449.4, 93538.6)_VehOdo_bin","[3799.2, 7598.4)_MMR_factor_bin","[1165.6, 1869.2)_WarrantyCost_bin"
2,OTHER_Auction,2_VehicleAge,DODGE_Make,SE_Trim,BLACK_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,MEDIUM_Size,CHRYSLER_TopThreeAmericanName,74135_VNZIP1,OK_VNST,"[7297.8, 10946.2)_VehBCost_bin","[38092.6, 49181.8)_VehOdo_bin","[7598.4, 11397.6)_MMR_factor_bin","[462.0, 1165.6)_WarrantyCost_bin"
3,OTHER_Auction,4_VehicleAge,FORD_Make,S_Trim,BROWN_Color,AUTO_Transmission,Covers_WheelType,AMERICAN_Nationality,VAN_Size,FORD_TopThreeAmericanName,85018_VNZIP1,AZ_VNST,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[3799.2, 7598.4)_MMR_factor_bin","[1165.6, 1869.2)_WarrantyCost_bin"
4,MANHEIM_Auction,4_VehicleAge,CHRYSLER_Make,Bas_Trim,BLUE_Color,AUTO_Transmission,Alloy_WheelType,AMERICAN_Nationality,VAN_Size,CHRYSLER_TopThreeAmericanName,75236_VNZIP1,TX_VNST,"[3649.4, 7297.8)_VehBCost_bin","[71360.2, 82449.4)_VehOdo_bin","[3799.2, 7598.4)_MMR_factor_bin","[1165.6, 1869.2)_WarrantyCost_bin"


In [19]:
#we have to pass the values of df_new as a list to run in the right way apriori algorithm
baskets = df_new.values.tolist()

In [20]:
#BASKET EXAMPLE
baskets[0]

['ADESA_Auction',
 '2_VehicleAge',
 'KIA_Make',
 'EX_Trim',
 'BLUE_Color',
 'AUTO_Transmission',
 'Covers_WheelType',
 'OTHER ASIAN_Nationality',
 'MEDIUM_Size',
 'OTHER_TopThreeAmericanName',
 '33619_VNZIP1',
 'FL_VNST',
 '[3649.4, 7297.8)_VehBCost_bin',
 '[60271.0, 71360.2)_VehOdo_bin',
 '[7598.4, 11397.6)_MMR_factor_bin',
 '[462.0, 1165.6)_WarrantyCost_bin']

In [21]:
from fim import apriori

In [22]:
#This is the explaination of the parameters of apriori
print(apriori.__doc__)

apriori (tracts, target='s', supp=10, zmin=1, zmax=None, report='a',
         eval='x', agg='x', thresh=10, prune=None, algo='b', mode='',
         border=None)
Find frequent item sets with the Apriori algorithm.
tracts  transaction database to mine (mandatory)
        The database must be an iterable of transactions;
        each transaction must be an iterable of items;
        each item must be a hashable object.
        If the database is a dictionary, the transactions are
        the keys, the values their (integer) multiplicities.
target  type of frequent item sets to find     (default: s)
        s/a   sets/all   all     frequent item sets
        c     closed     closed  frequent item sets
        m     maximal    maximal frequent item sets
        g     gens       generators
        r     rules      association rules
supp    minimum support of an item set         (default: 10)
        (positive: percentage, negative: absolute number)
conf    minimum confidence of an assoc. rul

In [23]:
rules = apriori(baskets, supp=30, zmin=2, target='r', conf=80, report='YXlC') 
print(len(rules))
contare = 0
for y in rules:
        print('%s -- %s' % (str(contare), y))
        contare = contare + 1

68
0 -- ('AMERICAN_Nationality', ('[71360.2, 82449.4)_VehOdo_bin',), 83.58515777572207, 30.424504060026724, 1.0068897191276371, 84.16103603603604)
1 -- ('AUTO_Transmission', ('[71360.2, 82449.4)_VehOdo_bin',), 96.49158872100593, 30.424504060026724, 1.0046154097142983, 96.93693693693693)
2 -- ('AUTO_Transmission', ('CHRYSLER_TopThreeAmericanName', 'AMERICAN_Nationality'), 96.49158872100593, 32.08791585294823, 1.0199825010619052, 98.41973199508836)
3 -- ('AMERICAN_Nationality', ('CHRYSLER_TopThreeAmericanName', 'AUTO_Transmission'), 83.58515777572207, 31.58084078528112, 1.1963846532218396, 100.0)
4 -- ('AMERICAN_Nationality', ('CHRYSLER_TopThreeAmericanName',), 83.58515777572207, 32.08791585294823, 1.1963846532218396, 100.0)
5 -- ('AUTO_Transmission', ('CHRYSLER_TopThreeAmericanName',), 96.49158872100593, 32.08791585294823, 1.0199825010619052, 98.41973199508836)
6 -- ('AUTO_Transmission', ('GM_TopThreeAmericanName', 'AMERICAN_Nationality'), 96.49158872100593, 34.681536300407714, 1.007795

In [24]:
#RULES EXAMPLE
#INDEX: 
# 0 : 'AMERICAN_Nationality'
# 1 : ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')
# 2 : 83.58515777572207 refers to 'Y' in report='YXlC'  that is relative head(head is 'AMERICAN_Nationality') item support as a percentage
# 3 : 20.27957652379484 refers to 'X' in report='YXlC'  that is X relative body(body is ('[82449.4, 93538.6)_VehOdo_bin', 'AUTO_Transmission')) set  support as a percentage
# 4 : 1.0436783505509324 refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
# 5 : 87.23601959790506 refers to 'C' in report='YXlC'  that is rule confidence as a percentage
rules[4]

('AMERICAN_Nationality',
 ('CHRYSLER_TopThreeAmericanName',),
 83.58515777572207,
 32.08791585294823,
 1.1963846532218396,
 100.0)

In [25]:
#Code that filters all the association rules with a certain lift under
count = 0
for r in rules:
    if r[4]>1.4: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
        print('%s -- %s' % (str(count), r))
        count = count + 1
print('Rules with high lift: {}'.format(count))

0 -- ('[3649.4, 7297.8)_VehBCost_bin', ('[3799.2, 7598.4)_MMR_factor_bin', 'AMERICAN_Nationality', 'AUTO_Transmission'), 59.31236509404872, 31.736732106759856, 1.5394683859392237, 91.30951095757314)
1 -- ('[3649.4, 7297.8)_VehBCost_bin', ('[3799.2, 7598.4)_MMR_factor_bin', 'AMERICAN_Nationality'), 59.31236509404872, 33.18772056052352, 1.5393472817593716, 91.30232798224333)
2 -- ('[3649.4, 7297.8)_VehBCost_bin', ('[3799.2, 7598.4)_MMR_factor_bin', 'AUTO_Transmission'), 59.31236509404872, 38.33042107787713, 1.530237599348255, 90.76201117318435)
3 -- ('[3649.4, 7297.8)_VehBCost_bin', ('[3799.2, 7598.4)_MMR_factor_bin',), 59.31236509404872, 40.545448316031106, 1.5305543951532286, 90.7808010816292)
Rules with high lift: 4


In [26]:
#TASKS ON KAGGLE:
# 1) Frequent patterns extraction with different values of support and different types (i.e. frequent, close, maximal), (6 points)
# 2) Discussion of the most interesting frequent patterns and analyze how changes the number of patterns w.r.t. the min_sup parameter (7 points)

In [27]:
# s/a   sets/all   all     frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='a') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 3911
Min support = 10 
 Itemsets = 800
Min support = 15 
 Itemsets = 293
Min support = 20 
 Itemsets = 135
Min support = 25 
 Itemsets = 71
Min support = 30 
 Itemsets = 48
Min support = 35 
 Itemsets = 22
Min support = 40 
 Itemsets = 14
Min support = 45 
 Itemsets = 10
Min support = 50 
 Itemsets = 3
Min support = 55 
 Itemsets = 2
Min support = 60 
 Itemsets = 1
Min support = 65 
 Itemsets = 1
Min support = 70 
 Itemsets = 1
Min support = 75 
 Itemsets = 1
Min support = 80 
 Itemsets = 1
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [28]:
# c     closed     closed  frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='c') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 2895
Min support = 10 
 Itemsets = 664
Min support = 15 
 Itemsets = 259
Min support = 20 
 Itemsets = 126
Min support = 25 
 Itemsets = 69
Min support = 30 
 Itemsets = 46
Min support = 35 
 Itemsets = 22
Min support = 40 
 Itemsets = 14
Min support = 45 
 Itemsets = 10
Min support = 50 
 Itemsets = 3
Min support = 55 
 Itemsets = 2
Min support = 60 
 Itemsets = 1
Min support = 65 
 Itemsets = 1
Min support = 70 
 Itemsets = 1
Min support = 75 
 Itemsets = 1
Min support = 80 
 Itemsets = 1
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [29]:
#m     maximal    maximal frequent item sets
for i in range(5,101,5):
    itemsets = apriori(baskets, supp=i, zmin=2, target='m') 
    print("Min support = {} \n Itemsets = {}".format(i,len(itemsets)))

Min support = 5 
 Itemsets = 644
Min support = 10 
 Itemsets = 170
Min support = 15 
 Itemsets = 80
Min support = 20 
 Itemsets = 40
Min support = 25 
 Itemsets = 23
Min support = 30 
 Itemsets = 20
Min support = 35 
 Itemsets = 11
Min support = 40 
 Itemsets = 7
Min support = 45 
 Itemsets = 5
Min support = 50 
 Itemsets = 3
Min support = 55 
 Itemsets = 2
Min support = 60 
 Itemsets = 1
Min support = 65 
 Itemsets = 1
Min support = 70 
 Itemsets = 1
Min support = 75 
 Itemsets = 1
Min support = 80 
 Itemsets = 1
Min support = 85 
 Itemsets = 0
Min support = 90 
 Itemsets = 0
Min support = 95 
 Itemsets = 0
Min support = 100 
 Itemsets = 0


In [30]:
#TASK ON KAGGLE:
# 3) Association rules extraction with different values of confidence (6 points)
# 4) Discussion of the most interesting rules and analyze how changes the number of rules w.r.t. the min_conf parameter, histogram of rules' confidence and lift (7 points)

In [34]:
#EXTRACT RULES
for i in range(10,101,10):
    for j in range(10,101,10):
        rules = apriori(baskets, supp=i, zmin=2, target='r', conf=j, report='YXlC') 
        print("Support = {} \nConfidence = {} \nRules = {}".format(i,j,len(rules)))
        #Code that filters all the association rules with a certain lift under
        count = 0
        for r in rules:
            if r[4]>1.4: #r[4] indeed refers to index 4 so as we have seen before index 4 refers to refers to 'l' in report='YXlC'  that is lift value of a rule (confidence/prior)
                count = count + 1
        print('Rules with high lift: {}'.format(count))

Support = 10 
Confidence = 10 
Rules = 30421
Rules with high lift: 5757
Support = 10 
Confidence = 20 
Rules = 16581
Rules with high lift: 3440
Support = 10 
Confidence = 30 
Rules = 8946
Rules with high lift: 1717
Support = 10 
Confidence = 40 
Rules = 5926
Rules with high lift: 1063
Support = 10 
Confidence = 50 
Rules = 3826
Rules with high lift: 856
Support = 10 
Confidence = 60 
Rules = 1968
Rules with high lift: 574
Support = 10 
Confidence = 70 
Rules = 1218
Rules with high lift: 273
Support = 10 
Confidence = 80 
Rules = 952
Rules with high lift: 151
Support = 10 
Confidence = 90 
Rules = 697
Rules with high lift: 110
Support = 10 
Confidence = 100 
Rules = 168
Rules with high lift: 46
Support = 20 
Confidence = 10 
Rules = 6081
Rules with high lift: 802
Support = 20 
Confidence = 20 
Rules = 3189
Rules with high lift: 412
Support = 20 
Confidence = 30 
Rules = 1692
Rules with high lift: 167
Support = 20 
Confidence = 40 
Rules = 1123
Rules with high lift: 113
Support = 20 
Con